# FINM 36702 Portfolio Credit Risk HW1
# Qi Wang

In [3]:
from scipy import stats, optimize, integrate
import numpy as np
import pandas as pd
from sigfig import round
from IPython.display import display
pd.options.display.float_format = '{:,.2g}'.format

# Question1

In [4]:
PD1 = 0.1
PD2 = 0.2
PD3 = 0.3
PDJ12 = 0.06
PDJ13 = 0.06
PDJ23 = 0.06

## Find the three values of correlation, rho12, rho13, and rho23

In [5]:
def rho(PD1, PD2, PDJ12):
    def pdj(rho, PD1, PD2, PDJ12):   
        a = stats.multivariate_normal(mean=[0,0], cov=[[1,rho],[rho,1]], allow_singular=True)
        pdj = a.cdf([stats.norm.ppf(PD1),stats.norm.ppf(PD2)])-PDJ12
        return pdj
    rho = optimize.fsolve(pdj, 0, args=(PD1, PD2, PDJ12))
    return rho[0]

In [6]:
print("rho12 is %.2g"%rho(PD1,PD2,PDJ12))

rho12 is 0.6


In [7]:
print("rho13 is %.2g"%rho(PD1,PD3,PDJ13))

rho13 is 0.43


In [8]:
print("rho23 is %.2g"%rho(PD2,PD3,PDJ23))

rho23 is 0


## Find the three values of default correlation, Corr[D1, D2], Corr[D1, D3], and Corr[D2, D3].

In [9]:
def default_corr(PD1, PD2, PDJ12):
    corr = (PDJ12 - PD1*PD2) / np.sqrt(PD1*(1.-PD1)*PD2*(1.-PD2))
    return corr

In [10]:
print("Corr[D1,D2] is %.2g"%default_corr(PD1,PD2,PDJ12))

Corr[D1,D2] is 0.33


In [11]:
print("Corr[D1,D3] is %.2g"%default_corr(PD1,PD3,PDJ13))

Corr[D1,D3] is 0.22


In [12]:
print("Corr[D2,D3] is %.2g"%default_corr(PD2,PD3,PDJ23))

Corr[D2,D3] is 0


# Question2

## State the three values of PDJ

In [13]:
PD1 = 0.1
PD2 = 0.1
PD3 = 0.1
rho12 = 0.4
rho13  = 0.5
rho23 = 0.6

In [14]:
def pdj(rho, PD1, PD2):
    a = stats.multivariate_normal(mean=[0,0], cov=[[1,rho],[rho,1]], allow_singular=True)
    pdj = a.cdf([stats.norm.ppf(PD1),stats.norm.ppf(PD2)]) 
    return pdj

In [15]:
print("PDJ12 is %.2g"%pdj(rho12, PD1, PD2))

PDJ12 is 0.027


In [16]:
print("PDJ13 is %.2g"%pdj(rho13, PD1, PD3))

PDJ13 is 0.032


In [17]:
print("PDJ23 is %.2g"%pdj(rho23, PD2, PD3))

PDJ23 is 0.039


## State the range of possible values for the probability that all three of the firms default. 

It is [0,0.1]. If all three of the firms do not default, the probability is 0. If PDJ is 0, the probability is 0.1=PD1=PD2=PD3.

## State the probability that all three default under the Gauss copula.  

In [18]:
def pdj3(rho_matrix, PD1, PD2, PD3):
    a = stats.multivariate_normal(mean=np.zeros(3),cov=rho_matrix, allow_singular=True)
    pdj3 = a.cdf([stats.norm.ppf(PD1),stats.norm.ppf(PD2), stats.norm.ppf(PD3)])
    return pdj3

In [20]:
rho_matrix = np.matrix([[1, 0.4, 0.5],[0.4,1,0.6],[0.5,0.6,1]])
print("The probability that all three default under the Gauss copula is %.2g"%pdj3(rho_matrix, PD1, PD2, PD3))

The probability that all three default under the Gauss copula is 0.016


# Question 3

In [63]:
pdj_matrix = [[0,0,0],[0,0,0],[0,0,0]]
A_prob = [[0,0.1],[0.1,0.5],[0.5,1]]
B_prob = [[0,0.2],[0.2,0.7],[0.7,1]]

In [64]:
def c_pdj_matrix(rho, lower_Z1, upper_Z1, lower_Z2, upper_Z2):
    a = stats.multivariate_normal(mean=[0,0], cov=[[1,rho],[rho,1]], allow_singular=True)
    def _a(x, y):
        return a.pdf([x, y])
    res = integrate.dblquad(_a, stats.norm.ppf(lower_Z1), stats.norm.ppf(upper_Z1),
                           lambda x: stats.norm.ppf(lower_Z2),lambda x: stats.norm.ppf(upper_Z2))
    return res[0]

In [66]:
for i in range(3):
    lower_Z1 = A_prob[i][0]
    upper_Z1 = A_prob[i][1]
    for j in range(3):
        lower_Z2 = B_prob[j][0]
        upper_Z2 = B_prob[j][1]
        pdj_matrix[2-j][i] =  "{:.2f}".format(c_pdj_matrix(0.4, lower_Z1, upper_Z1, lower_Z2, upper_Z2))
pdj_matrix

[['0.01', '0.08', '0.21'], ['0.05', '0.21', '0.24'], ['0.04', '0.10', '0.06']]

In [67]:
pdj_matrix = pd.DataFrame(pdj_matrix, index=['FirmB=A','FirmB=B','FirmB=D'],
                                    columns=['FirmA=D','FirmA=B','FirmA=A'])
pdj_matrix

,FirmA=D,FirmA=B,FirmA=A
FirmB=A,0.01,0.08,0.21
FirmB=B,0.05,0.21,0.24
FirmB=D,0.04,0.10,0.06


# Question4

In [68]:
PD = [0.01,0.02,0.03,0.04]
PDJ = 0.001

In [70]:
rho_matrix = []
for i in range(4):
    row = []
    for j in range(4):
        if i ==j: row.append(1)
        else:
            row.append(rho(PD[i], PD[j], PDJ))
    rho_matrix.append(row)     

In [71]:
rho_matrix = pd.DataFrame(rho_matrix)
rho_matrix

,0,1,2,3
0,1,0.31,0.24,0.18
1,0.31,1,0.1,0.044
2,0.24,0.1,1,-0.036
3,0.18,0.044,-0.036,1


In [76]:
np.all(np.linalg.eigvals(rho_matrix) > 0)

True

The matrix of correlations is positive definite. Thus, it cannot be connected by a Gauss copula. 